# Classification with Delira and TorchScript - A very short introduction
*Author: Justus Schock* 

*Date: 04.12.2018*

This Example shows how to set up a basic classification `TorchScript` model and experiment.
`TorchScript` is basically `PyTorch` with a static computation graph. Thus, we require only minor changes compared to the `PyTorch`-example. These changes will be highlighted.

Let's first setup the essential hyperparameters. We will use `delira`'s `Parameters`-class for this:

In [ ]:
logger = None
import torch
from delira.training import Parameters
params = Parameters(fixed_params={
    "model": {
        "in_channels": 1, 
        "n_outputs": 10
    },
    "training": {
        "batch_size": 64, # batchsize to use
        "num_epochs": 10, # number of epochs to train
        "optimizer_cls": torch.optim.Adam, # optimization algorithm to use
        "optimizer_params": {'lr': 1e-3}, # initialization parameters for this algorithm
        "losses": {"CE": torch.nn.CrossEntropyLoss()}, # the loss function
        "lr_sched_cls": None,  # the learning rate scheduling algorithm to use
        "lr_sched_params": {}, # the corresponding initialization parameters
        "metrics": {} # and some evaluation metrics
    }
}) 

Since we did not specify any metric, only the `CrossEntropyLoss` will be calculated for each batch. Since we have a classification task, this should be sufficient. We will train our network with a batchsize of 64 by using `Adam` as optimizer of choice.

## Logging and Visualization
To get a visualization of our results, we should monitor them somehow. For logging we will use `Tensorboard`. Per default the logging directory will be the same as our experiment directory.


## Data Preparation
### Loading
Next we will create some fake data. For this we use the `ClassificationFakeData`-Dataset, which is already implemented in `deliravision`. To avoid getting the exact same data from both datasets, we use a random offset.

In [ ]:
from deliravision.data.fakedata import ClassificationFakeData
dataset_train = ClassificationFakeData(num_samples=10000, 
                                       img_size=(3, 224, 224), 
                                       num_classes=10)
dataset_val = ClassificationFakeData(num_samples=1000, 
                                     img_size=(3, 224, 224), 
                                     num_classes=10,
                                     rng_offset=10001
                                     )

### Augmentation
For Data-Augmentation we will apply a few transformations:

In [ ]:
from batchgenerators.transforms import RandomCropTransform, \
                                        ContrastAugmentationTransform, Compose
from batchgenerators.transforms.spatial_transforms import ResizeTransform
from batchgenerators.transforms.sample_normalization_transforms import MeanStdNormalizationTransform

transforms = Compose([
    RandomCropTransform(200), # Perform Random Crops of Size 200 x 200 pixels
    ResizeTransform(224), # Resample these crops back to 224 x 224 pixels
    ContrastAugmentationTransform(), # randomly adjust contrast
    MeanStdNormalizationTransform(mean=[0.5], std=[0.5])]) 



With these transformations we can now wrap our datasets into datamanagers:

In [ ]:
from delira.data_loading import BaseDataManager, SequentialSampler, RandomSampler

manager_train = BaseDataManager(dataset_train, params.nested_get("batch_size"),
                                transforms=transforms,
                                sampler_cls=RandomSampler,
                                n_process_augmentation=4)

manager_val = BaseDataManager(dataset_val, params.nested_get("batch_size"),
                              transforms=transforms,
                              sampler_cls=SequentialSampler,
                              n_process_augmentation=4)


## Model

After we have done that, we can specify our model: We will use a smaller version of a [VGG11](https://arxiv.org/pdf/1409.1556.pdf) in this case. We will use more convolutions to reduce the feature dimensionality and reduce the number of units in the linear layers to save up memory (and we only have to deal with 10 classes, not the 1000 imagenet classes).

In [2]:
from delira.models import AbstractTorchScriptNetwork
import torch

class Flatten(torch.nn.Module):
        
    def forward(self, x):
        return x.view(x.size(0), -1)

class VGG11TorchScript(AbstractTorchScriptNetwork):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.model = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, 64, 3, padding=1), # 224 x 224
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2), # 112 x 112
            torch.nn.Conv2d(64, 128, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2), # 56 x 56
            torch.nn.Conv2d(128, 256, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(256, 256, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2), # 28 x 28
            torch.nn.Conv2d(256, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(), # 14 x 14
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(), # 7 x 7
            torch.nn.Conv2d(512, 512, 3, padding=((1, 0), (1, 0))), # 6 x 6
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(), # 3 x 3
            torch.nn.Conv2d(512, 512, 3), # 1 x 1
            torch.nn.ReLU(),
            Flatten(),
            torch.nn.Linear(1*1*512, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(256, num_classes)
        )
        
    @torch.jit.script_method    
    def forward(self, x: torch.Tensor):
        return {"pred": self.model(x)}
    
    @staticmethod
    def prepare_batch(data_dict, input_device, output_device):
        return_dict = {"data": torch.from_numpy(batch["data"]).to(
            input_device).to(torch.float)}

        for key, vals in batch.items():
            if key == "data": 
                continue
            return_dict[key] = torch.from_numpy(vals).to(output_device).to(
                torch.float)

        return return_dict
    
    @staticmethod
    def closure(model, data_dict: dict, optimizers: dict, losses: dict,
                fold=0, **kwargs):

        loss_vals = {}
        total_loss = 0


        # predict
        inputs = data_dict["data"]
        preds = model(inputs)

        # calculate losses
        for key, crit_fn in losses.items():
            _loss_val = crit_fn(preds["pred"], data_dict["label"])
            loss_vals[key] = _loss_val.item()
            total_loss += _loss_val

        optimizers['default'].zero_grad()
        total_loss.backward()
        optimizers['default'].step()

        return loss_vals, {k: v.detach()
                                for k, v in preds.items()}
    
    

So let's evisit, what we have just done.

In `delira` all networks must be derived from `delira.models.AbstractNetwork`. For each backend there is a class derived from this class, handling some backend-specific function calls and registrations. For the `TorchScript` Backend this class is `AbstractTorchScriptNetwork` and all TorchScript Networks should be derived from it.

> **Note:** This is different from `PyTorch`, where the base class has to be `AbstractPyTorchNetwork`

First we defined the network itself (this is the part simply concatenating the layers into a sequential model). Next, we defined the logic to apply, when we want to predict from the model (this is the `forward` method).

> **Note:** In `TorchScript` all methods adding options to the computation graph must be decorated with `torch.jit.script_method`. See [here](https://pytorch.org/docs/stable/jit.html#creating-torchscript-code) for more details

So far this was plain `TorchScript`. The `prepare_batch` function is not plain TorchScript anymore, but allows us to ensure the data is in the correct shape, has the correct data-type and lies on the correct device. The function above is the standard `prepare_batch` function, which is also implemented in the `AbstractTorchScriptNetwork` and just re-implemented here for the sake of completeness.

Same goes for the `closure` function. This function defines the update rule for our parameters (and how to calculate the losses). These funcitons are good to go for many simple networks but can be overwritten for customization when training more complex networks.

## Training
Now that we have defined our network, we can finally specify our experiment and run it.

In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning) # ignore UserWarnings raised by dependency code
warnings.simplefilter("ignore", FutureWarning) # ignore FutureWarnings raised by dependency code


from delira.training import TorchScriptExperiment
from delira.training.train_utils import create_optims_default_pytorch

if logger is not None:
    logger.info("Init Experiment")
experiment = TorchScriptExperiment(params, SmallTorchScript,
                                   name="ClassificationExample",
                                   save_path="./tmp/delira_Experiments",
                                   optim_builder=create_optims_default_pytorch,
                                   key_mapping={"x": "data"}
                                   gpu_ids=[0])
experiment.save()

model = experiment.run(manager_train, manager_val)

Congratulations, you have now trained your first Classification Model using `delira`, we will now predict a few samples from the testset to show, that the networks predictions are valid:

In [ ]:
import numpy as np
from tqdm.auto import tqdm # utility for progress bars

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # set device (use GPU if available)
model = model.to(device) # push model to device
preds, labels = [], []

with torch.no_grad():
    for i in tqdm(range(len(dataset_val))):
        img = dataset_val[i]["data"] # get image from current batch
        img_tensor = torch.from_numpy(img).unsqueeze(0).to(device).to(torch.float) # create a tensor from image, push it to device and add batch dimension
        pred_tensor = model(img_tensor) # feed it through the network
        pred = pred_tensor.argmax(1).item() # get index with maximum class confidence
        label = np.asscalar(dataset_val[i]["label"]) # get label from batch
        if i % 1000 == 0:
            print("Prediction: %d \t label: %d" % (pred, label)) # print result
        preds.append(pred)
        labels.append(label)
        
# calculate accuracy
accuracy = (np.asarray(preds) == np.asarray(labels)).sum() / len(preds)
print("Accuracy: %.3f" % accuracy)